In [ ]:
pip install tokenizers

     |████████████████████████████████| 2.9MB 8.6MB/s 


In [ ]:
from tokenise import BPE_token
from pathlib import Path
import os
# the folder 'text' contains all the files
paths = [str(x) for x in Path("./data/").glob("**/*.txt")]
tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(paths)
# saving the tokenized data in our specified folder 
save_path = 'tokenized_data'
tokenizer.save_tokenizer(save_path)

In [ ]:
pip install transformers

     |████████████████████████████████| 1.4MB 8.0MB/s 
     |████████████████████████████████| 890kB 32.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c8508e77a76dac8a92cc43e1da0c3bc3f94d92f6d09fd4c9e9e0728020b640e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import pandas as pd
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [ ]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
   single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [ ]:
print(string_tokenized)

[28, 277, 3115, 78, 4194, 51, 3223, 6398, 80, 1281, 55, 3000, 530, 84, 78, 6, 7985, 897, 183, 51, 2103, 530, 740, 84, 223, 6, 192, 762, 7, 75, 91, 119, 598, 80, 1898, 55, 8432, 183, 55, 1761, 5, 20784, 740, 8963, 2103, 4546, 391, 185, 486, 488, 4471, 7, 1731, 8, 1317, 7, 11263, 8, 38, 15754, 6, 24938, 366, 20593, 6, 12, 169, 232, 319, 5, 112, 51, 900, 1793, 7, 55, 459, 2821, 91, 109, 5, 507, 171, 110, 409, 24650, 2679, 55, 1950, 84, 55, 8432, 1147, 78, 55, 740, 5, 110, 247, 120, 103, 931, 80, 1507, 303, 1403, 125, 14415, 227, 55, 8432, 1147, 84, 55, 740, 1889, 227, 55, 2988, 49, 1551, 213, 2369, 240, 564, 895, 2731, 326, 329, 3058, 5, 95, 7, 26, 1199, 49, 749, 213, 3184, 329, 7744, 305, 1228, 7, 242, 49, 55, 727, 433, 2174, 125, 3684, 5, 110, 406, 80, 514, 303, 55, 49, 2653, 49, 1041, 84, 1202, 23419, 91, 7, 227, 9978, 55, 2653, 1041, 112, 6248, 5329, 91, 1321, 3513, 5, 199, 55, 3190, 84, 2653, 91, 2057, 6768, 112, 24266, 5329, 49, 887, 119, 394, 2205, 80, 103, 3385, 78, 55, 1180, 242,

In [ ]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
dataset

<BatchDataset shapes: ((12, 99), (12, 99)), types: (tf.int32, tf.int32)>

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
num_epoch = 50
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/50
169/309 [===============>..............] - ETA: 33s - loss: 2.4037 - logits_loss: 2.4037 - logits_accuracy: 0.5675 - past_key_values_1_accuracy: 0.0033 - past_key_values_2_accuracy: 0.0032 - past_key_values_3_accuracy: 0.0034 - past_key_values_4_accuracy: 0.0030 - past_key_values_5_accuracy: 0.0035 - past_key_values_6_accuracy: 0.0033 - past_key_values_7_accuracy: 0.0031 - past_key_values_8_accuracy: 0.0038 - past_key_values_9_accuracy: 0.0034 - past_key_values_10_accuracy: 0.0031 - past_key_values_11_accuracy: 0.0031 - past_key_values_12_accuracy: 0.0034

In [38]:
text = "i am interested in creating a web tool to predict the absolute risk of in-hospital death from a published risk model of covid-19 patients. is it possible to estimate the intercept from the significant, multivariable model coefficients published table 3 at http://doi.org/10.1016/s0140-6736 20 30566-3 answer "
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 500,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


In [39]:
print(tokenizer.decode(beam_output[0]))

i am interested in creating a web tool to predict the absolute risk of in-hospital death from a published risk model of covid-19 patients. is it possible to estimate the intercept from the significant, multivariable model coefficients published table 3 at http://doi.org/10.1016/s0140-6736 20 30566-3 answer  you could also, that, you would not make a small model, the best way to buy a few in the end of the actual problem with, even if that means that can use a single exponential model could make sure you can not provide in case fatality rate - after getting infected person has done into account the above the same time is because the exponential growth rate to make sense. if you might have been made a large number of getting sick person with a very small amount of course this means in your small rate, they need to reduce the epidemic, for example here, such as many cases which means out an epidemic cases based on the population model will be less common cold, because people who infected 

In [44]:
predicted_ans = tokenizer.decode(beam_output[0])
start = predicted_ans.find('answer')
predicted_ans= predicted_ans[start+8:]

In [45]:
test = "no, for a few reasons. the most relevant is that, even if you could numerically determine the value of the intercept term in the model, you would not be able to tell what conditions are represented by the intercept term of the model fit by the researchers unless they provided some very specific details about their methods, e.g., did they center their predictors , etc. . the way these models are defined, you need to know what the reference point of each coefficient is. by convention the reference point for continuous variables is often zero, but the choice of reference is completely arbitrary for categorical variables though it does appear to be stated in the article . on a more conceptual note, models like this one are often limited in their ability to predict accurate class probabilities because their goal is to explain relationships between predictors, or to describe relative risk-reducing or risk-increasing relationships among the predictors in the model. often these models use only a limited number of variables to arrive at a parsimonious model, which tries to keep multicollinearity low, so that the individual effects of predictors are easier to interpret. another way of looking at it is this: the model may have been designed to address research questions such as: what is the relative importance of age with respect to the risk of an in-hospital death after controlling for other demographic factors . even if the model does this very well, it may be very bad at answering fundamentally different questions, such as what is the probability that a female will experience an in-hospital death"

In [47]:
import nltk
bleu_score =[]
bleu_score.append(nltk.translate.bleu_score.sentence_bleu(test, predicted_ans))

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [48]:
bleu_score

[0.33572726329452635]